# NL2SQL leveraging the Azure Accelerator

In [1]:
# This code block imports necessary libraries and modules for connecting to databases, processing data, 
# and utilizing the LangChain framework for SQL generation from natural language. It also sets up 
# environment variables and defines a function for Markdown display in Jupyter notebooks.
import os
import pandas as pd
import pyodbc
from langchain_openai import AzureChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent, SQLDatabaseToolkit
# from langchain.agents import create_sql_agent 
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain.agents import create_sql_agent  # No longer in agent_toolkits
from langchain.sql_database import SQLDatabase  # Toolkit is replaced with SQLDatabase
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager
from common.prompts import MSSQL_AGENT_PREFIX
from IPython.display import Markdown, HTML, display  
from dotenv import load_dotenv

# load secrets from .env file
load_dotenv("credentials.env")

# Function to print markdown comments
def printmd(string):
    display(Markdown(string))

printmd("##### Depedencies loaded...")

# Set environment variables
app_env = os.environ["AZURE_OPENAI_API_VERSION"]

printmd(f"app_env: {app_env}")

##### Depedencies loaded...

app_env: 2023-12-01-preview

In [2]:
# This code block sets up logging for the application. 
# It defines a custom log format that includes the timestamp, 
# logger name, log level, and the log message. # 
import logging

# Define a custom log format
log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'

# logging.basicConfig(level=logging.DEBUG)
# logging.basicConfig(level=logging.DEBUG, format=log_format)

logging.basicConfig(level=logging.INFO, format=log_format)

logger = logging.getLogger('langchain')

printmd("##### Logger created...")

##### Logger created...

In [3]:
# Connect to Azure SQL Database using SQLAlchemy and pyodbc. 
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
import os

# Configuration for the database connection
db_config = {
    'drivername': 'mssql+pyodbc',
    'username': os.environ["SQL_SERVER_USERNAME"] + '@' + os.environ["SQL_SERVER_NAME"],
    'password': os.environ["SQL_SERVER_PASSWORD"],
    'host': os.environ["SQL_SERVER_NAME"],
    'port': 1433,
    'database': os.environ["SQL_SERVER_DATABASE"],
    'query': {'driver': 'ODBC Driver 17 for SQL Server'},
}

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

# Connect to the Azure SQL Database using the URL string
engine = create_engine(db_url)

# Test the connection using the SQLAlchemy 2.0 execution style
with engine.connect() as conn:
    try:
        # Use the text() construct for safer SQL execution
        result = conn.execute(text("SELECT @@VERSION"))
        version = result.fetchone()
        print("Connection successful!")
        print(version)
    except Exception as e:
        print(e)

# Create SQLDatabase class. 
# For Langchain, it connectd and interactd with SQL databases.
# It serves as an interface to perform operations like querying data
# and fetching results, integrating with the language model.
db = SQLDatabase.from_uri(db_url)

# Test the connection to the database by fetching the table names
print("SqlAgent Created. Found following tables:")
print(db.get_usable_table_names())

printmd(f"Done!")

Connection successful!
('Microsoft SQL Azure (RTM) - 12.0.2000.8 \n\tJun 19 2024 16:01:48 \n\tCopyright (C) 2022 Microsoft Corporation\n',)
SqlAgent Created. Found following tables:
['Artists', 'Conditions', 'Descriptions', 'Genres', 'Mediums', 'Products', 'Status', 'sysdiagrams']


Done!

In [ ]:
# # Initialize instance of AzureChatOpenAI 
# llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=2000, api_version=os.environ["AZURE_OPENAI_API_VERSION"])
# printmd(f"LLM Created: {llm.deployment_name}")

# # Create SQLDatabase class. 
# # For Langchain, it connectd and interactd with SQL databases.
# # It serves as an interface to perform operations like querying data
# # and fetching results, integrating with the language model.
# db = SQLDatabase.from_uri(db_url)

# # Test the connection to the database by fetching the table names
# print("SqlAgent Created. Found following tables:")
# print(db.get_usable_table_names())

# Pick a Prompt from the following selection

In [ ]:
# Natural Language question (query)
QUESTION = """
How may patients were hospitalized during July 2020 in Texas. 
And nationwide as the total of all states? 
Use the hospitalizedIncrease column
"""
printmd(f"{QUESTION}")

In [4]:
# Natural Language question (query)
QUESTION = """
How many products are available in the inventory?
"""
printmd(f"{QUESTION}")


How many products are available in the inventory?


In [ ]:
# Natural Language question (query)
QUESTION = """
List the all of the rock products
"""
printmd(f"{QUESTION}")

In [ ]:
# Natural Language question (query)
QUESTION = """
What products include Bruce Springsteen?
"""
printmd(f"{QUESTION}")

In [5]:
# Initialize instance of AzureChatOpenAI 
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=2000, api_version=os.environ["AZURE_OPENAI_API_VERSION"])
printmd(f"LLM Created: {llm.deployment_name}")

# system_prompt = f"Given the following SQL database schema, write a SQL query to answer the following question:"
system_prompt = (
    "You are a LangChain SQL agent designed to generate accurate SQL statements from natural language text inputs. "
    "Your primary responsibilities include:\n\n"
    "- Parsing natural language queries to identify key components such as tables, columns, conditions, and operations.\n"
    "- Generating well-formed SQL statements that precisely match the intent and constraints described in the input text.\n"
    "- Handling various SQL operations including SELECT, INSERT, UPDATE, DELETE, and complex joins.\n"
    "- Validating the generated SQL against the provided database schema to ensure correctness."
)

# SQLDatabaseToolkit is a utility for interacting with a SQL database using a language model (LLM).
# It facilitate the integration between the database and the language model, enabling more sophisticated
# query generation, execution, and result handling.
# Parameters in the SQLDatabase class and the LLM.
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Create the SqlAgent
# SqlAgent interacts with directly with the SQL database. 
# It leverages the LLM to generate SQL queries from natural language input
# and then execute these queries on the connected database. 
# Note how the SqlAgent leverges the SQLDatabaseToolkit and the language model (LLM). 
# try:
    # agent_executor = create_sql_agent(
    #     prefix=MSSQL_AGENT_PREFIX,
    #     llm=llm,
    #     toolkit=toolkit,
    #     top_k=30,
    #     agent_type="openai-tools",
    #     verbose=True,
    #     agent_executor_kwargs={"return_intermediate_steps": True},
    #     system_prompt=system_prompt  # Step 2: Pass the system prompt to the AgentExecutor
    # )
try:
    sql_agent = create_sql_agent(
        prefix=MSSQL_AGENT_PREFIX,
        llm=llm,
        toolkit=toolkit,
        top_k=30,
        agent_type="openai-tools",
        verbose=True,
        agent_executor_kwargs={"return_intermediate_steps": True},
        system_prompt=system_prompt  # Step 2: Pass the system prompt to the AgentExecutor
    )
except Exception as e:
    print(f"An error occurred while creating the agent_executor: {str(e)}")
    sql_agent = None  # Set agent_executor to None or handle it as needed

# Continue with the rest of your code, checking if agent_executor was created successfully
if sql_agent:
    printmd(f"##### Agent Executor Created")

# Langchain agents use expert/tools. List the tools for SQL Agent
sql_agent.tools

printmd(f"Complete")

LLM Created: gpt4-o

##### Agent Executor Created

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000016851E56A50>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000016851E56A50>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000016851E56A50>),
 QuerySQLCheckerTool(description='Use this tool to 

In [7]:
show_details = False

try:
    response = sql_agent.invoke(QUESTION) 
except Exception as e:
    response = str(e)

# Advanced logging block
# Block iterates through the intermediate steps of the response 
# and prints the action and observation for each step.
# This is a common pattern for displaying detailed information based on 
# the content of data structures and conditional flag in Python
# Importantly, the block reveals the agent's decision-making process.
if show_details: 
    # Iterate through each step in the 'intermediate_steps' list from the response
    for step in response['intermediate_steps']:
         # Check if the current step is a dictionary
        if isinstance(step, dict):
            # Extract 'action' and 'observation' from the step, defaulting to empty string if not found
            action = step.get('action', '')
            observation = step.get('observation', '')
        # Check if the current step is a tuple (assumed to be a pair of action and observation)
        elif isinstance(step, tuple):
            # Unpack the tuple directly into action and observation variables
            action, observation = step 
        else:
            # Handle unexpected step types by printing a warning and skipping the current iteration
            print(f"Unexpected step type: {type(step)}")
            continue  # Skip this step if it's neither dict nor tuple
        # Print the action and observation for the current step
        print(f"Action: {action}\nObservation: {observation}\n")
# else:
#     # If the flag to show details is not set, indicate that there are no details to show
#     print("No details to show")

# print(response['intermediate_steps'])



> Entering new SQL Agent Executor chain...


2024-07-16 19:25:16,537 - httpx - INFO - HTTP Request: POST https://ai-west-openai.openai.azure.com//openai/deployments/gpt4-o/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"



Invoking: `sql_db_list_tables` with `{}`


Artists, Conditions, Descriptions, Genres, Mediums, Products, Status, sysdiagrams

2024-07-16 19:25:17,040 - httpx - INFO - HTTP Request: POST https://ai-west-openai.openai.azure.com//openai/deployments/gpt4-o/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"



Invoking: `sql_db_schema` with `{'table_names': 'Products'}`



CREATE TABLE [Products] (
	[Id] INTEGER NOT NULL IDENTITY(1,1), 
	[ProductId] UNIQUEIDENTIFIER NOT NULL, 
	[GenreId] INTEGER NOT NULL, 
	[ArtistId] INTEGER NOT NULL, 
	[StatusId] INTEGER NOT NULL, 
	[MediumId] INTEGER NOT NULL, 
	[ConditionId] INTEGER NOT NULL, 
	[Title] NVARCHAR(160) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Single] NVARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[AlbumArtUrl] NVARCHAR(160) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Price] DECIMAL(18, 2) NOT NULL, 
	[Cost] DECIMAL(18, 2) NOT NULL, 
	[ParentalCaution] BIT NOT NULL DEFAULT ((0)), 
	[UPC] NCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[ReleaseYear] NVARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[HighValueItem] BIT NOT NULL, 
	[CreateDate] DATETIME2 NOT NULL DEFAULT (getdate()), 
	[RemoveDate] DATETIME2 NULL, 
	[IsActive] BIT NOT NULL DEFAULT ((1)), 
	CONSTRAINT [PK_Products] PRIMARY KEY ([Id]), 

...

```sql
SELECT COUNT(*) AS TotalProducts FROM Products WHERE IsActive = 1
```

2024-07-16 19:25:19,524 - httpx - INFO - HTTP Request: POST https://ai-west-openai.openai.azure.com//openai/deployments/gpt4-o/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"



Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) AS TotalProducts FROM Products WHERE IsActive = 1'}`


[(185,)]

2024-07-16 19:25:20,357 - httpx - INFO - HTTP Request: POST https://ai-west-openai.openai.azure.com//openai/deployments/gpt4-o/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"


Final Answer: There are 185 products available in the inventory.

Explanation:
I queried the `Products` table to count the number of products where the `IsActive` column is set to 1, indicating that the product is available in the inventory. The SQL query used is:

```sql
SELECT COUNT(*) AS TotalProducts FROM Products WHERE IsActive = 1
```
This query returned a count of 185 products.

> Finished chain.


In [8]:
# Final print statement
print (response["output"])

Final Answer: There are 185 products available in the inventory.

Explanation:
I queried the `Products` table to count the number of products where the `IsActive` column is set to 1, indicating that the product is available in the inventory. The SQL query used is:

```sql
SELECT COUNT(*) AS TotalProducts FROM Products WHERE IsActive = 1
```
This query returned a count of 185 products.
